# Urban dashboard

## Listes villes 

+ Cameroun
 + Yaoundé
 +    Douala
 + Bamenda
 + Bafoussam
 + Ngaoundéré
 + Nkongsamba
+ République Dominicaine
 +    Saint Domingue
+ Indonésie
 +    Bandung
+ Maroc
 + Fès
 + Rabat/ Salé
 + Oujda
 + Agadir
 + Kenitra
 + Beni Melal
 + Marrakech
 + Khourigba
 + Ej Jadida
 + Settat
 + Safi
 + Khenisset
 + Laayoune
 + Meknes
 + Tanger
 


### Phase 1: Get coordinates from a city list 

In [16]:
 maroc = ["fès", "rabat", "oujda", "agadir", "kenitra", "beni melal","marrakech","Khourigba","Ej Jadida","Settat","Safi","Khenisset","Laayoune","Meknes"," Tanger"]

cameroun = ["Yaoundé", "Douala", "Bamenda", "Bafoussam", "Ngaoundéré", "Nkongsamba"]

indonesie = ["Jakarta", "Surabaya", "Bandung"]

rep_dom = ["Saint Domingue"] 

liste_pays = {"Maroc":maroc, "Cameroun": cameroun, "Indonésie":indonesie, "République Dominicaine": rep_dom}

import geocoder


coo = {}
bbox = {}
for i in liste_pays:
    for j in liste_pays[i]:
        name= j+'_'+i
        g = geocoder.google(name)
        t =g.latlng
        coo[name] =  t
        bbox[name] = [t[0]-0.15, t[1]-0.15, t[0]+0.15, t[1]+0.15]
        


### Phase 2: Do OSM request and create corresponding JSON

In [21]:
import overpass
from string import Template


places_list = ['hospital', 'marketplace','uni','townhall']
api = overpass.API()
dict_query = {'hospital' : '''(
      node["amenity"="hospital"]$bbox;
      way["amenity"="hospital"]$bbox;
      relation["amenity"="hospital"]$bbox;
    )''' ,  'uni' : '''(
      node["amenity"="university"]$bbox;
      way["amenity"="university"]$bbox;
      relation["amenity"="university"]$bbox;
    )''' , 'townhall' : ''' (
      node["amenity"="townhall"]$bbox;
      way["amenity"="townhall"]$bbox;
      relation["amenity"="townhall"]$bbox;
    )
    ''' , 'marketplace' :'''
    ( node["amenity"="marketplace"]$bbox;
      way["amenity"="marketplace"]$bbox;
      relation["amenity"="marketplace"]$bbox;
    )
    '''}

for i in bbox:
    temp = str(bbox[i])
    
    bb = "("+temp[1:-1]+")"

    Input = {'bbox': bb}



    for z in dict_query:
        t = Template(dict_query[z]).safe_substitute(Input)
        response= api.Get(t)
        nam = i.replace(' ','')
        print(nam)
        Input_save = {'query' : z, 'nam' : nam}
        name = Template('$nam-$query.geojson').safe_substitute(Input_save)
        print(name)
        with open('data/'+name, "w") as f: f.write(str(response))


Bafoussam_Cameroun
Bafoussam_Cameroun-hospital.geojson
Bafoussam_Cameroun
Bafoussam_Cameroun-uni.geojson
Bafoussam_Cameroun
Bafoussam_Cameroun-townhall.geojson
Bafoussam_Cameroun
Bafoussam_Cameroun-marketplace.geojson
kenitra_Maroc
kenitra_Maroc-hospital.geojson
kenitra_Maroc
kenitra_Maroc-uni.geojson
kenitra_Maroc
kenitra_Maroc-townhall.geojson
kenitra_Maroc
kenitra_Maroc-marketplace.geojson
Laayoune_Maroc
Laayoune_Maroc-hospital.geojson
Laayoune_Maroc
Laayoune_Maroc-uni.geojson
Laayoune_Maroc
Laayoune_Maroc-townhall.geojson
Laayoune_Maroc
Laayoune_Maroc-marketplace.geojson
Settat_Maroc
Settat_Maroc-hospital.geojson
Settat_Maroc
Settat_Maroc-uni.geojson
Settat_Maroc
Settat_Maroc-townhall.geojson
Settat_Maroc
Settat_Maroc-marketplace.geojson
Bamenda_Cameroun
Bamenda_Cameroun-hospital.geojson
Bamenda_Cameroun
Bamenda_Cameroun-uni.geojson
Bamenda_Cameroun
Bamenda_Cameroun-townhall.geojson
Bamenda_Cameroun
Bamenda_Cameroun-marketplace.geojson
Khenisset_Maroc
Khenisset_Maroc-hospital.geojs

### Phase 3: What if we create HTML files now ?

In [22]:

html = ''' <!DOCTYPE html>
<html>

<head>
    <title> City Dashboard - $name</title>

    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

    <link rel="stylesheet" href="https://npmcdn.com/leaflet@1.0.0-rc.3/dist/leaflet.css" />
    <script src="https://npmcdn.com/leaflet@1.0.0-rc.3/dist/leaflet.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-ajax/2.1.0/leaflet.ajax.min.js"></script>


    <style>
        html,
        body {
            height: 100%;
        }
        
        #map {
            width: 100%;
            height: 100%;
        }
        
        .info {
            padding: 6px 8px;
            font: 14px/16px Arial, Helvetica, sans-serif;
            background: white;
            background: rgba(255, 255, 255, 0.8);
            box-shadow: 0 0 15px rgba(0, 0, 0, 0.2);
            border-radius: 5px;
        }
        
        .info h4 {
            margin: 0 0 5px;
            color: #777;
        }
        
        .legend {
            text-align: left;
            line-height: 18px;
            color: #555;
        }
        
        .legend i {
            width: 18px;
            height: 18px;
            float: left;
            margin-right: 8px;
            opacity: 0.7;
        }

    </style>
</head>

<body>

    <div id="map"></div>
    <script type="text/javascript">
        var map = L.map('map', {
            center: $center,
            zoom: 13,
            minZoom: 12
        });
        L.tileLayer('https://api.mapbox.com/styles/v1/etibdv/cj0kv2vnc003a2rt8m01gpf4h/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZXRpYmR2IiwiYSI6ImNpejhvdWJmcjAwMW8yd28weTkzMnA1aDkifQ.i8UKq0M_sIN1qq8F6UAgFw', {
            maxZoom: 18,
            attribution: 'Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors, ' +
            '<a href="http://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, ' +
            'Dataviz from <a href="https://twitter.com/EtienneDavid">@EtienneDavid</a> with Python',
            id: 'mapbox.light'
        }).addTo(map);

        function onEachFeature(feature, layer) {
            var popupContent = "Type: " + feature.properties.amenity + "</br> Name: " +
            feature.properties.name;

            if (feature.properties && feature.properties.popupContent) {
                popupContent += feature.properties.popupContent;
            }

            layer.bindPopup(popupContent);
        }


        var style_hospital = {
            "color": "#f00000",
            "backgroundColor": "#f00000",
            "weight": 5,
            "opacity": 0.65
        };

        var hospitalIcon = L.icon({
            iconUrl: 'icon/hospital.png',
            iconSize: [32, 32],
            iconAnchor: [16, 37],
            popupAnchor: [0, -28]
        });

        var geojsonLayer_hospital = new L.GeoJSON.AJAX("data/$name-hospital.geojson", {
            style: style_hospital,
            onEachFeature: onEachFeature,

            pointToLayer: function(feature, latlng) {
                return L.marker(latlng, {
                    icon: hospitalIcon
                });
            }
        });
        geojsonLayer_hospital.addTo(map);

        var style_uni = {
            "color": "#12a81c",
            "weight": 5,
            "opacity": 0.65
        };

        var geojsonLayer_uni = new L.GeoJSON.AJAX("data/$name-uni.geojson", {
            onEachFeature: onEachFeature,
            style: style_uni
        });


        geojsonLayer_uni.addTo(map);

        var geojsonLayer_townhall = new L.GeoJSON.AJAX("data/$name-townhall.geojson", {
            onEachFeature: onEachFeature
        });
        geojsonLayer_townhall.addTo(map)

        var style_marketplace = {
            "color": "#e9e223",
            "weight": 5,
            "opacity": 0.65
        };

        var geojsonLayer_marketplace = new L.GeoJSON.AJAX("data/$name-marketplace.geojson", {
            onEachFeature: onEachFeature,
            style: style_marketplace
        });


        geojsonLayer_marketplace.addTo(map)

        L.control.scale({"imperial": false, "maxWidth" : 200}).addTo(map);
        console.log(map.getBounds());

    </script>
</body>

</html>'''

for i in coo:
    name =i.replace(' ','')
    Input = {'center' : coo[i], 'name' : name}
    maps = Template(html).safe_substitute(Input)
    with open(name+'.html', "w") as f: f.write(maps)
